# Generate Written Marketing Content Grounded with your Message and Positioning Framework (MPF)

This requires building a Knowledge Graph from your MPF in Neo4j. You can use the Neo4j Labs [LLM Graph Builder](https://github.com/neo4j-labs/llm-graph-builder) to accomplish this.

In [1]:
SOURCE_MATERIAL_TXT_FILE = 'transcript.txt'
OUTPUT_FILE = 'summary.txt'
NUM_VERSIONS = 4
TASK_PROMPT = """
You are an expert Marketer for Neo4j, the Graph Database and Analytics company.
You must create a Youtube title and description for our new GenAI Hero Demo using the Source Transcript below. The description should be engaging and encourage people to want to learn more.
Also, below the Source Transcript is the Messaging Framework which you should use as context to guide the formation of the title and description. The Messaging Framework will tell you about the strengths of Neo4j to highlight as we as the target audience you are writing for.

# Source Transcript
{taskMaterials}

# Messaging Framework
{messagingFramework}
"""


In [2]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(TASK_PROMPT)

In [3]:
# You can skip this cell if not using a ws.env file - alternative to above
from dotenv import load_dotenv
import os

load_dotenv('.env', override=True)

# Neo4j
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
AURA_DS = eval(os.getenv('AURA_DS').title())

# AI
LLM = 'gpt-4'

In [4]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
embedding_dimension = 1536

In [5]:
# Import relevant libraries
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

#Instantiate LLM
llm = ChatOpenAI(temperature=0.7, model_name=LLM, streaming=True)

In [6]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

neo4jStore = Neo4jVector.from_existing_index(
        embedding=embedding_model,
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        index_name='vector',
        retrieval_query="""
        WITH node AS chunk, score
        MATCH (chunk)-[:PART_OF]->(doc)
        OPTIONAL MATCH (chunk)<-[:NEXT_CHUNK]-(pc)
        OPTIONAL MATCH (chunk)-[:NEXT_CHUNK]->(nc)
        RETURN coalesce(pc.text,'') + chunk.text + coalesce(nc.text,'') AS text,
            score,
            {source: doc.fileName} AS metadata
        ORDER BY score DESC
    """)


In [7]:
from langchain_core.runnables import RunnablePassthrough

kg_retriever = neo4jStore.as_retriever(search_kwargs={"k": 10})

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = ({'messagingFramework': kg_retriever | format_docs,
          'taskMaterials':  RunnablePassthrough()}
         | prompt
         | llm
         | StrOutputParser())

In [8]:
with open(SOURCE_MATERIAL_TXT_FILE, "r") as f:
    txt = f.read()

In [9]:
response = chain.invoke(txt)

In [10]:
from tqdm import tqdm

with open(OUTPUT_FILE, "a") as out_txt_file:
    for i in tqdm(range(NUM_VERSIONS)):
        out_txt_file.write(f'#Response {i}\n')
        out_txt_file.write(chain.invoke(txt))
        out_txt_file.write('\n\n')

100%|██████████| 4/4 [00:36<00:00,  9.24s/it]
